In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

2.1.0-dev20191124


In [5]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [6]:
# Define a simple sequential model
def create_model():
    model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Creating checkpoints

checkpoint_path = "training_1/cp.ckpt"
checkpoit_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)

model.fit(train_images,
         train_labels,
         epochs=10,
         validation_data=(test_images,test_labels),
         callbacks=[cp_callback] # Passing callback to training
         )


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 608/1000 [=================>............] - ETA: 0s - loss: 1.4147 - accuracy: 0.5921 
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 1ms/sample - loss: 1.1368 - accuracy: 0.6740 - val_loss: 0.7199 - val_accuracy: 0.7800
Epoch 2/10
 736/1000 [=====================>........] - ETA: 0s - loss: 0.4165 - accuracy: 0.8859
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 146us/sample - loss: 0.3999 - accuracy: 0.8900 - val_loss: 0.5134 - val_accuracy: 0.8390
Epoch 3/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.2846 - accuracy: 0.9232
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 118us/sample - loss: 0.2694 - accuracy: 0.9270 - val_loss: 0.4735 - val_accuracy: 0.8410
Epoch 4/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.1996 - accuracy: 0.9488
Epoch 

In [9]:
!ls {checkpoit_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [10]:
# Ubtrained Model

model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained Model, accuracy: {}%".format(100*acc))

1000/1000 - 0s - loss: 2.4170 - accuracy: 0.0720
Untrained Model, accuracy: 7.199999690055847%


In [11]:
# Loading model weights
model.load_weights(checkpoint_path)

# Re-evaluating the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored Model, accuracy: {}%".format(100*acc))

1000/1000 - 0s - loss: 0.3969 - accuracy: 0.8700
Restored Model, accuracy: 87.00000047683716%


# Playing with Checkpoint Callbacks

In [12]:
# Saving weights after every 5 epochs
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Creating a callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5
    )

# Creating model instance
model = create_model()

# Saving model weights
model.save_weights(checkpoint_path.format(epoch=0))

#Training the model with new callback
model.fit(
    train_images,
    train_labels,
    epochs=50,
    callbacks=[cp_callback],
    validation_data=(test_images, test_labels),
    verbose=0
            )






Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [18]:
# finding the latest entry of model
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

training_2/cp-0050.ckpt


In [19]:
# test the model by loading from latest checkpoint

model = create_model()
model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored Model's Accuracy: {}".format(100*acc))

1000/1000 - 0s - loss: 0.4877 - accuracy: 0.8730
Restored Model's Accuracy: 87.30000257492065


## Using HDF5

In [20]:
# Create and train a new model instance
model = create_model()
model.fit(train_images, train_labels, epochs=5)

model.save('mymodel.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 238us/sample - loss: 1.1497 - accuracy: 0.6710
Epoch 2/5
1000/1000 [==============================] - 0s 74us/sample - loss: 0.4209 - accuracy: 0.8780
Epoch 3/5
1000/1000 [==============================] - 0s 74us/sample - loss: 0.2848 - accuracy: 0.9360
Epoch 4/5
1000/1000 [==============================] - 0s 74us/sample - loss: 0.2125 - accuracy: 0.9520
Epoch 5/5
1000/1000 [==============================] - 0s 68us/sample - loss: 0.1501 - accuracy: 0.9710


In [21]:
# Recreating the model from that file

new_model = tf.keras.models.load_model('mymodel.h5')

new_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [22]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 - 0s - loss: 0.4407 - accuracy: 0.8580
Restored model, accuracy: 85.80%
